In [1]:
import os, sys, glob, datetime

# specify spark version, python version
spark_home = "/home/zero/spark-2.4.0-bin-hadoop2.7" # MODIFY THIS
python_path="/apps/anaconda3/bin/python"
# set environment variables
os.environ['SPARK_HOME'] = spark_home
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['SPARK_LOCAL_IP'] = "127.0.0.1"

def setup_spark_env(app_name):
    # set environment variables
    spark_python = os.path.join(spark_home, 'python')
    py4j = glob.glob(os.path.join(spark_python, 'lib', 'py4j-*.zip'))[0]
    sys.path[:0] = [spark_python, py4j]
    # specify Spark application parameters
    PYSPARK_SUBMIT_ARGS="--master local[2]"

    os.environ['PYSPARK_SUBMIT_ARGS'] = (PYSPARK_SUBMIT_ARGS 
        + " --name '%s_%s'"%(app_name, datetime.datetime.now().strftime("%Y%m%d %H:%M")) 
        + " pyspark-shell")    
    return

#
setup_spark_env("your_spark_process_name") # MODIFY THIS
# launching PySpark application
# execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))
filename=os.path.join(spark_home, 'python/pyspark/shell.py')
exec(compile(open(filename, "rb").read(), filename, 'exec'))
sc.setLogLevel('ERROR')
print("{}".format(sc.applicationId))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.0
      /_/

Using Python version 3.6.4 (default, Jan 16 2018 18:10:19)
SparkSession available as 'spark'.
local-1557024762906


In [2]:
from pyspark.sql import functions as sf
from pyspark.sql import Row
from pyspark.sql.types import *
import numpy as np

In [3]:
import os, math, subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# some settings for displaying Pandas results
pd.set_option('display.width', 2000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.precision', 4)
pd.set_option('display.max_colwidth', -1)

In [4]:
# load data
data_path = "home-credit-default-risk/application_train.csv"
df = sqlContext.read.format("csv").option("header", "true").load(data_path)
print(df.take(1))

[Row(SK_ID_CURR='100002', TARGET='1', NAME_CONTRACT_TYPE='Cash loans', CODE_GENDER='M', FLAG_OWN_CAR='N', FLAG_OWN_REALTY='Y', CNT_CHILDREN='0', AMT_INCOME_TOTAL='202500.0', AMT_CREDIT='406597.5', AMT_ANNUITY='24700.5', AMT_GOODS_PRICE='351000.0', NAME_TYPE_SUITE='Unaccompanied', NAME_INCOME_TYPE='Working', NAME_EDUCATION_TYPE='Secondary / secondary special', NAME_FAMILY_STATUS='Single / not married', NAME_HOUSING_TYPE='House / apartment', REGION_POPULATION_RELATIVE='0.018801', DAYS_BIRTH='-9461', DAYS_EMPLOYED='-637', DAYS_REGISTRATION='-3648.0', DAYS_ID_PUBLISH='-2120', OWN_CAR_AGE=None, FLAG_MOBIL='1', FLAG_EMP_PHONE='1', FLAG_WORK_PHONE='0', FLAG_CONT_MOBILE='1', FLAG_PHONE='1', FLAG_EMAIL='0', OCCUPATION_TYPE='Laborers', CNT_FAM_MEMBERS='1.0', REGION_RATING_CLIENT='2', REGION_RATING_CLIENT_W_CITY='2', WEEKDAY_APPR_PROCESS_START='WEDNESDAY', HOUR_APPR_PROCESS_START='10', REG_REGION_NOT_LIVE_REGION='0', REG_REGION_NOT_WORK_REGION='0', LIVE_REGION_NOT_WORK_REGION='0', REG_CITY_NOT_LI

In [5]:
total_records = df.count()
print("total_records:", total_records)

total_records: 307511


In [6]:
# check dtypes
for n, t in df.dtypes:
    print("{} ({})".format(n, t))

SK_ID_CURR (string)
TARGET (string)
NAME_CONTRACT_TYPE (string)
CODE_GENDER (string)
FLAG_OWN_CAR (string)
FLAG_OWN_REALTY (string)
CNT_CHILDREN (string)
AMT_INCOME_TOTAL (string)
AMT_CREDIT (string)
AMT_ANNUITY (string)
AMT_GOODS_PRICE (string)
NAME_TYPE_SUITE (string)
NAME_INCOME_TYPE (string)
NAME_EDUCATION_TYPE (string)
NAME_FAMILY_STATUS (string)
NAME_HOUSING_TYPE (string)
REGION_POPULATION_RELATIVE (string)
DAYS_BIRTH (string)
DAYS_EMPLOYED (string)
DAYS_REGISTRATION (string)
DAYS_ID_PUBLISH (string)
OWN_CAR_AGE (string)
FLAG_MOBIL (string)
FLAG_EMP_PHONE (string)
FLAG_WORK_PHONE (string)
FLAG_CONT_MOBILE (string)
FLAG_PHONE (string)
FLAG_EMAIL (string)
OCCUPATION_TYPE (string)
CNT_FAM_MEMBERS (string)
REGION_RATING_CLIENT (string)
REGION_RATING_CLIENT_W_CITY (string)
WEEKDAY_APPR_PROCESS_START (string)
HOUR_APPR_PROCESS_START (string)
REG_REGION_NOT_LIVE_REGION (string)
REG_REGION_NOT_WORK_REGION (string)
LIVE_REGION_NOT_WORK_REGION (string)
REG_CITY_NOT_LIVE_CITY (string)
REG_C

In [7]:
# count distinct
for cname in df.columns:
    cnt_dist = df.select(cname).distinct().count()
    pct_dist = cnt_dist * 100.0 / total_records
    print("{}: {} ({:0.2f}%)".format(cname, cnt_dist, pct_dist))

SK_ID_CURR: 307511 (100.00%)
TARGET: 2 (0.00%)
NAME_CONTRACT_TYPE: 2 (0.00%)
CODE_GENDER: 3 (0.00%)
FLAG_OWN_CAR: 2 (0.00%)
FLAG_OWN_REALTY: 2 (0.00%)
CNT_CHILDREN: 15 (0.00%)
AMT_INCOME_TOTAL: 2548 (0.83%)
AMT_CREDIT: 5603 (1.82%)
AMT_ANNUITY: 13673 (4.45%)
AMT_GOODS_PRICE: 1003 (0.33%)
NAME_TYPE_SUITE: 8 (0.00%)
NAME_INCOME_TYPE: 8 (0.00%)
NAME_EDUCATION_TYPE: 5 (0.00%)
NAME_FAMILY_STATUS: 6 (0.00%)
NAME_HOUSING_TYPE: 6 (0.00%)
REGION_POPULATION_RELATIVE: 81 (0.03%)
DAYS_BIRTH: 17460 (5.68%)
DAYS_EMPLOYED: 12574 (4.09%)
DAYS_REGISTRATION: 15688 (5.10%)
DAYS_ID_PUBLISH: 6168 (2.01%)
OWN_CAR_AGE: 63 (0.02%)
FLAG_MOBIL: 2 (0.00%)
FLAG_EMP_PHONE: 2 (0.00%)
FLAG_WORK_PHONE: 2 (0.00%)
FLAG_CONT_MOBILE: 2 (0.00%)
FLAG_PHONE: 2 (0.00%)
FLAG_EMAIL: 2 (0.00%)
OCCUPATION_TYPE: 19 (0.01%)
CNT_FAM_MEMBERS: 18 (0.01%)
REGION_RATING_CLIENT: 3 (0.00%)
REGION_RATING_CLIENT_W_CITY: 3 (0.00%)
WEEKDAY_APPR_PROCESS_START: 7 (0.00%)
HOUR_APPR_PROCESS_START: 24 (0.01%)
REG_REGION_NOT_LIVE_REGION: 2 (0.00%)

In [8]:
# count NULL
for cname in df.columns:
    cnt_null = df.where("{} is NULL".format(cname)).count()
    pct_miss = cnt_null * 100.0 / total_records
    print("{}: {} ({:0.2f}%)".format(cname, cnt_null, pct_miss))

SK_ID_CURR: 0 (0.00%)
TARGET: 0 (0.00%)
NAME_CONTRACT_TYPE: 0 (0.00%)
CODE_GENDER: 0 (0.00%)
FLAG_OWN_CAR: 0 (0.00%)
FLAG_OWN_REALTY: 0 (0.00%)
CNT_CHILDREN: 0 (0.00%)
AMT_INCOME_TOTAL: 0 (0.00%)
AMT_CREDIT: 0 (0.00%)
AMT_ANNUITY: 12 (0.00%)
AMT_GOODS_PRICE: 278 (0.09%)
NAME_TYPE_SUITE: 1292 (0.42%)
NAME_INCOME_TYPE: 0 (0.00%)
NAME_EDUCATION_TYPE: 0 (0.00%)
NAME_FAMILY_STATUS: 0 (0.00%)
NAME_HOUSING_TYPE: 0 (0.00%)
REGION_POPULATION_RELATIVE: 0 (0.00%)
DAYS_BIRTH: 0 (0.00%)
DAYS_EMPLOYED: 0 (0.00%)
DAYS_REGISTRATION: 0 (0.00%)
DAYS_ID_PUBLISH: 0 (0.00%)
OWN_CAR_AGE: 202929 (65.99%)
FLAG_MOBIL: 0 (0.00%)
FLAG_EMP_PHONE: 0 (0.00%)
FLAG_WORK_PHONE: 0 (0.00%)
FLAG_CONT_MOBILE: 0 (0.00%)
FLAG_PHONE: 0 (0.00%)
FLAG_EMAIL: 0 (0.00%)
OCCUPATION_TYPE: 96391 (31.35%)
CNT_FAM_MEMBERS: 2 (0.00%)
REGION_RATING_CLIENT: 0 (0.00%)
REGION_RATING_CLIENT_W_CITY: 0 (0.00%)
WEEKDAY_APPR_PROCESS_START: 0 (0.00%)
HOUR_APPR_PROCESS_START: 0 (0.00%)
REG_REGION_NOT_LIVE_REGION: 0 (0.00%)
REG_REGION_NOT_WORK_REG

In [9]:
# count zeros
for cname in df.columns:
    cnt_zeros = df.where("{} = 0.0".format(cname)).count()
    pct_zeros = cnt_zeros * 100.0 / total_records
    print("{}: {} ({:0.2f}%)".format(cname, cnt_zeros, pct_zeros))

SK_ID_CURR: 0 (0.00%)
TARGET: 282686 (91.93%)
NAME_CONTRACT_TYPE: 0 (0.00%)
CODE_GENDER: 0 (0.00%)
FLAG_OWN_CAR: 0 (0.00%)
FLAG_OWN_REALTY: 0 (0.00%)
CNT_CHILDREN: 215371 (70.04%)
AMT_INCOME_TOTAL: 0 (0.00%)
AMT_CREDIT: 0 (0.00%)
AMT_ANNUITY: 0 (0.00%)
AMT_GOODS_PRICE: 0 (0.00%)
NAME_TYPE_SUITE: 0 (0.00%)
NAME_INCOME_TYPE: 0 (0.00%)
NAME_EDUCATION_TYPE: 0 (0.00%)
NAME_FAMILY_STATUS: 0 (0.00%)
NAME_HOUSING_TYPE: 0 (0.00%)
REGION_POPULATION_RELATIVE: 0 (0.00%)
DAYS_BIRTH: 0 (0.00%)
DAYS_EMPLOYED: 2 (0.00%)
DAYS_REGISTRATION: 80 (0.03%)
DAYS_ID_PUBLISH: 16 (0.01%)
OWN_CAR_AGE: 2134 (0.69%)
FLAG_MOBIL: 1 (0.00%)
FLAG_EMP_PHONE: 55386 (18.01%)
FLAG_WORK_PHONE: 246203 (80.06%)
FLAG_CONT_MOBILE: 574 (0.19%)
FLAG_PHONE: 221080 (71.89%)
FLAG_EMAIL: 290069 (94.33%)
OCCUPATION_TYPE: 0 (0.00%)
CNT_FAM_MEMBERS: 0 (0.00%)
REGION_RATING_CLIENT: 0 (0.00%)
REGION_RATING_CLIENT_W_CITY: 0 (0.00%)
WEEKDAY_APPR_PROCESS_START: 0 (0.00%)
HOUR_APPR_PROCESS_START: 40 (0.01%)
REG_REGION_NOT_LIVE_REGION: 302854 

In [10]:
# count negative
for cname in df.columns:
    cnt_neg = df.where("{} < 0".format(cname)).count()
    pct_neg = cnt_neg * 100.0 / total_records
    print("{}: {} ({:0.2f}%)".format(cname, cnt_neg, pct_neg))

SK_ID_CURR: 0 (0.00%)
TARGET: 0 (0.00%)
NAME_CONTRACT_TYPE: 0 (0.00%)
CODE_GENDER: 0 (0.00%)
FLAG_OWN_CAR: 0 (0.00%)
FLAG_OWN_REALTY: 0 (0.00%)
CNT_CHILDREN: 0 (0.00%)
AMT_INCOME_TOTAL: 0 (0.00%)
AMT_CREDIT: 0 (0.00%)
AMT_ANNUITY: 0 (0.00%)
AMT_GOODS_PRICE: 0 (0.00%)
NAME_TYPE_SUITE: 0 (0.00%)
NAME_INCOME_TYPE: 0 (0.00%)
NAME_EDUCATION_TYPE: 0 (0.00%)
NAME_FAMILY_STATUS: 0 (0.00%)
NAME_HOUSING_TYPE: 0 (0.00%)
REGION_POPULATION_RELATIVE: 0 (0.00%)
DAYS_BIRTH: 307511 (100.00%)
DAYS_EMPLOYED: 252135 (81.99%)
DAYS_REGISTRATION: 307431 (99.97%)
DAYS_ID_PUBLISH: 307495 (99.99%)
OWN_CAR_AGE: 0 (0.00%)
FLAG_MOBIL: 0 (0.00%)
FLAG_EMP_PHONE: 0 (0.00%)
FLAG_WORK_PHONE: 0 (0.00%)
FLAG_CONT_MOBILE: 0 (0.00%)
FLAG_PHONE: 0 (0.00%)
FLAG_EMAIL: 0 (0.00%)
OCCUPATION_TYPE: 0 (0.00%)
CNT_FAM_MEMBERS: 0 (0.00%)
REGION_RATING_CLIENT: 0 (0.00%)
REGION_RATING_CLIENT_W_CITY: 0 (0.00%)
WEEKDAY_APPR_PROCESS_START: 0 (0.00%)
HOUR_APPR_PROCESS_START: 0 (0.00%)
REG_REGION_NOT_LIVE_REGION: 0 (0.00%)
REG_REGION_NOT_

In [11]:
# stats for number
ls_features = [cname for cname in df.columns if cname != "SK_ID_CURR"]
pdf_stats = df.select(ls_features).describe().toPandas()
pdf_stats.T

,0,1,2,3,4
summary,count,mean,stddev,min,max
TARGET,307511,0.08072881945686496,0.27241864564839546,0,1
NAME_CONTRACT_TYPE,307511,None,None,Cash loans,Revolving loans
CODE_GENDER,307511,None,None,F,XNA
FLAG_OWN_CAR,307511,None,None,N,Y
FLAG_OWN_REALTY,307511,None,None,N,Y
CNT_CHILDREN,307511,0.4170517477423572,0.722121384437626,0,9
AMT_INCOME_TOTAL,307511,168797.91929698453,237123.14627885324,100071.0,99900.0
AMT_CREDIT,307511,599025.9997057016,402490.77699585445,1000417.5,999886.5
AMT_ANNUITY,307499,27108.573909183444,14493.737315118291,100017.0,9999.0
